In [ ]:
# Copyright 2022 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# 使用BigQuery ML进行在线预测

<table align="left">

  <td>
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/bigquery_ml/bqml-online-prediction.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> 在Colab中运行
    </a>
  </td>
  <td>
    <a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/bigquery_ml/bqml-online-prediction.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      在GitHub上查看
    </a>
  </td>
  <td>
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/vertex-ai-samples/main/notebooks/official/bigquery_ml/bqml-online-prediction.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo">
      在Vertex AI Workbench中打开
    </a>
  </td>                                                                                               
</table>

**_注意_**: 这个笔记本已经在以下环境中进行过测试：

* Python 版本 = 3.9

## 概览

本笔记本演示了如何在Google Cloud Platform上使用BigQuery ML训练客户流失预测模型。客户流失模型的训练是在Google Analytics 4应用事件数据上进行的，旨在估计应用程序中用户流失的可能性。

从博客中了解更多关于用例和数据的信息：[使用Google Analytics 4（GA4）和BigQuery ML进行游戏开发者流失预测](https://cloud.google.com/blog/topics/developers-practitioners/churn-prediction-game-developers-using-google-analytics-4-ga4-and-bigquery-ml)。

在训练BigQuery ML模型之后，本笔记本还探讨了在Vertex AI中注册受过训练的模型，并部署到端点以进行在线预测。

### 目标

在本教程中，您将从公共BigQuery数据集中获取所需数据并准备进行训练。然后，您将使用BigQuery ML在数据上训练一个流失预测模型。最后，您将部署模型到Vertex AI并从端点获取预测结果。

本教程使用以下Google Cloud数据分析和机器学习服务：

- BigQuery
- BigQuery ML
- Vertex AI模型注册
- Vertex AI端点

执行的步骤包括：

- 从公共BigQuery数据集中查询并获取数据。
- 准备数据进行训练。
- 使用BigQuery ML训练流失分类模型。
- 将训练好的模型保存到Vertex AI模型注册。
- 将模型部署到Vertex AI端点。
- 向端点发送在线预测请求。

### 数据集

这份笔记本使用了公开的BigQuery数据集 [`firebase-public-project.analytics_153293282`](https://console.cloud.google.com/bigquery?p=firebase-public-project&d=analytics_153293282&t=events_20181003&page=table)，其中包含来自一个名为Flood It!（安卓应用、iOS应用）的真实移动游戏应用的原始事件数据。数据架构源自于Google Analytics for Firebase，但与Google Analytics 4拥有相同的架构。因此，这份笔记本中的步骤可以应用于Google Analytics for Firebase或Google Analytics 4的数据。

Google Analytics 4（GA4）采用了基于事件的测量模型。事件提供关于应用程序或网站上正在发生的情况的见解，例如用户动作、系统事件或错误。数据中的每一行都是一个事件，具有关于该事件的各种相关特征存储在行内的嵌套格式中。

了解更多关于[数据集](https://cloud.google.com/blog/topics/developers-practitioners/churn-prediction-game-developers-using-google-analytics-4-ga4-and-bigquery-ml)。

### 成本

本教程使用Google Cloud的计费组件：

- BigQuery
- BigQuery ML
- Vertex AI

了解有关<a href="https://cloud.google.com/bigquery/pricing" target="_blank">BigQuery定价</a>，<a href="https://cloud.google.com/bigquery-ml/pricing" target="_blank">BigQuery ML定价</a>，<a href="https://cloud.google.com/vertex-ai/pricing" target="_blank">Vertex AI定价</a>，并使用<a href="https://cloud.google.com/products/calculator/" target="_blank">定价计算器</a>根据您的预期使用情况生成成本估算。

## 安装

安装以下所需的包以运行这个笔记本。

In [ ]:
! pip3 install --upgrade google-cloud-aiplatform \
                                 google-cloud-bigquery==3.11.4 \
                                 db-dtypes

只有Colab：请取消注释以下单元格以重新启动内核

In [ ]:
# Automatically restart kernel after installs so that your environment can access the new packages
# import IPython

# app = IPython.Application.instance()
# app.kernel.do_shutdown(True)

## 开始之前

### 设置你的 Google Cloud 项目

**无论你使用什么笔记本环境，以下步骤都是必需的。**

1. [选择或创建一个 Google Cloud 项目](https://console.cloud.google.com/cloud-resource-manager)。当你第一次创建账户时，你会获得$300的免费信用用于计算/存储费用。

2. [确保为您的项目启用了计费](https://cloud.google.com/billing/docs/how-to/modify-project)。

3. [启用 Vertex AI API](https://console.cloud.google.com/flows/enableapi?apiid=aiplatform.googleapis.com)。

4. 如果你是在本地运行这个笔记本，你需要安装[Cloud SDK](https://cloud.google.com/sdk)。

设置您的项目ID

**如果您不知道您的项目ID**，请尝试以下操作：
* 运行 `gcloud config list`。
* 运行 `gcloud projects list`。
* 查看支持页面：[查找项目ID](https://support.google.com/googleapi/answer/7014113)

In [ ]:
PROJECT_ID = "[your-project-id]"  # @param {type:"string"}

# Set the project id
! gcloud config set project {PROJECT_ID}

#### 区域

您也可以更改 Vertex AI 使用的 `REGION` 变量。了解更多关于 [Vertex AI 区域](https://cloud.google.com/vertex-ai/docs/general/locations)。

In [ ]:
REGION = "us-central1"  # @param {type: "string"}

### 验证您的Google Cloud账户

根据您的Jupyter环境，您可能需要手动进行身份验证。请按照以下相关说明操作。

**1. Vertex AI Workbench**
* 不需要进行任何操作，因为您已经通过验证。

**2. 本地JupyterLab实例，取消注释并运行：**

In [ ]:
# ! gcloud auth login

3. 合作，取消注释并运行：

In [ ]:
# from google.colab import auth
# auth.authenticate_user()

请参考如何在https://cloud.google.com/storage/docs/gsutil/commands/iam#ch-examples为您的服务帐户授予Cloud Storage权限。

创建一个云存储桶

创建一个存储桶，用作Vertex AI的暂存桶。

In [ ]:
BUCKET_URI = f"gs://your-bucket-name-{PROJECT_ID}-unique"  # @param {type:"string"}

只要您的存储桶尚未存在：运行以下单元格以创建您的云存储存储桶。

In [ ]:
! gsutil mb -l {REGION} -p {PROJECT_ID} {BUCKET_URI}

### 导入库

In [ ]:
import json
import os
from typing import Union

import google.cloud.aiplatform as vertex_ai
import pandas as pd
from google.cloud import bigquery

初始化 Python 的 Vertex AI 和 BigQuery SDK。

为您的项目初始化 Vertex AI SDK 并创建 BigQuery 客户端。

In [ ]:
# initialize the vertex ai sdk
vertex_ai.init(project=PROJECT_ID, location=REGION, staging_bucket=BUCKET_URI)

# create the bigquery client object
bq_client = bigquery.Client(project=PROJECT_ID)

在BigQuery中创建一个数据集之前,请定义一个辅助函数用于在BigQuery中运行查询。

In [ ]:
# Wrapper to use BigQuery client to run query/job, return job ID or result as DF


def run_bq_query(sql: str) -> Union[str, pd.DataFrame]:
    """
    Input: SQL query, as a string, to execute in BigQuery
    Returns the query results as a pandas DataFrame, or error, if any
    """
    # Try dry run before executing query to catch any errors
    job_config = bigquery.QueryJobConfig(dry_run=True, use_query_cache=False)
    bq_client.query(sql, job_config=job_config)

    # If dry run succeeds without errors, proceed to run query
    job_config = bigquery.QueryJobConfig()
    client_result = bq_client.query(sql, job_config=job_config)

    job_id = client_result.job_id

    # Wait for query/job to finish running. then get & return data frame
    df = client_result.result().to_arrow().to_pandas()
    print(f"Finished job_id: {job_id}")
    # return the dataframe (if any tabular data is returned)
    return df

使用SQL中的`Create`语句创建一个BigQuery数据集。

了解有关[BigQuery中的SQL方言](https://cloud.google.com/bigquery/docs/introduction-sql#bigquery-sql-dialects)的更多信息。

In [ ]:
# provide the dataset name
BQ_DATASET_NAME = "churnprediction_unique"  # @param {type:"string"}

# create a SQL query for creating the dataset
create_dataset_query = (
    f"""CREATE SCHEMA IF NOT EXISTS `{PROJECT_ID}.{BQ_DATASET_NAME}`"""
)

# run the query
run_bq_query(create_dataset_query)

## 准备训练数据

在训练机器学习模型之前，需要处理数据并从中提取适当的特征。

事件数据还包括用户的一些行为数据，这些数据可以用于生成训练特征。从数据中处理时间戳信息，以确定用户是否流失，这将作为训练的基本事实。

在本节中，您执行以下步骤：

1. 根据事件时间戳信息确定用户是否流失。
2. 提取每个用户的行为数据。
3. 将行为数据与流失标签数据进行匹配。

注意：原始[博客](https://cloud.google.com/blog/topics/developers-practitioners/churn-prediction-game-developers-using-google-analytics-4-ga4-and-bigquery-ml)关于对该数据集进行训练还考虑了用户人口统计数据，本教程中略去了该部分。

### 为每个用户标识标签

事件数据集没有一个特征来告诉你用户是否“流失”或“返回”。因此，在这一部分，根据以下标准为每个用户创建这个标签：

```
如果用户在首次使用应用程序后的24小时内没有显示任何事件数据，则认为该用户已流失。
```

用户流失可以有很多定义方式。在本笔记本中，您定义为1天的流失。

此外，您从时间戳中提取日历信息作为特征，并从数据中删除“跳动”情况。跳动情况指的是用户只花几分钟（比如10分钟）与应用程序交互的情况。

运行以下单元格以创建一个名为 **`returningusers`** 的视图，包括以下列：

- `user_pseudo_id`：用户的伪id。
- `user_first_engagement`：用户的最早事件时间戳。
- `user_last_engagement`：用户的最新事件时间戳。
- `month`：用户首次参与的年份月份。
- `julianday`：用户首次参与的年份天数。
- `dayofweek`：用户首次参与的星期几。
- `ts_24hr_after_first_engagement`：用户首次参与后24小时的时间戳。
- `churned`：布尔字段，表示已流失的为 **1**，未流失的为 **0**。
- `bounced`：布尔字段，表示已跳动的为 **1**，未跳动的为 **0**。

In [ ]:
# define the sql query
create_label_data_query = f"""
    CREATE OR REPLACE VIEW `{PROJECT_ID}.{BQ_DATASET_NAME}.returningusers` AS (
      WITH firstlasttouch AS (
        SELECT
          user_pseudo_id,
          MIN(event_timestamp) AS user_first_engagement,
          MAX(event_timestamp) AS user_last_engagement
        FROM
          `firebase-public-project.analytics_153293282.events_*`
        WHERE event_name="user_engagement"
        GROUP BY
          user_pseudo_id

      )
      SELECT
        user_pseudo_id,
        user_first_engagement,
        user_last_engagement,
        EXTRACT(MONTH from TIMESTAMP_MICROS(user_first_engagement)) as month,
        EXTRACT(DAYOFYEAR from TIMESTAMP_MICROS(user_first_engagement)) as julianday,
        EXTRACT(DAYOFWEEK from TIMESTAMP_MICROS(user_first_engagement)) as dayofweek,

        #add 24 hr to user's first touch
        (user_first_engagement + 86400000000) AS ts_24hr_after_first_engagement,

    #churned = 1 if last_touch within 24 hr of app installation, else 0
    IF (user_last_engagement < (user_first_engagement + 86400000000),
        1,
        0 ) AS churned,

    #bounced = 1 if last_touch within 10 min, else 0
    IF (user_last_engagement <= (user_first_engagement + 600000000),
        1,
        0 ) AS bounced,
      FROM
        firstlasttouch
      GROUP BY
        1,2,3
        );

    SELECT 
      * 
    FROM 
      `{PROJECT_ID}.{BQ_DATASET_NAME}.returningusers`
    LIMIT 10;
"""

# run the query
run_bq_query(create_label_data_query)

提取每个用户的行为数据

原始事件数据中的行为数据跨越多个事件。因此，您需要聚合和提取每个用户的行为数据。

由于模型需要根据用户在头24小时内的活动来预测，所以您需要从应用使用的数据中提取少于24小时的聚合数据。

对于聚合，您需要统计每个用户在数据中出现的以下事件类型的总数：

- 用户参与
- 开始快速游玩关卡
- 结束快速游玩关卡
- 完成快速游玩关卡
- 重置快速游玩关卡
- 发布分数
- 消耗虚拟货币
- 广告奖励
- 向朋友挑战
- 完成5个关卡
- 使用额外步骤

运行以下单元格来创建一个名为 `user_aggregate_behavior` 的视图，查询聚合的行为数据。

In [ ]:
create_behavioral_data_query = f"""
    CREATE OR REPLACE VIEW `{PROJECT_ID}.{BQ_DATASET_NAME}.user_aggregate_behavior` AS (
    WITH
      events_first24hr AS (
        SELECT
          e.*
        FROM
          `firebase-public-project.analytics_153293282.events_*` e
        JOIN
          `{PROJECT_ID}.{BQ_DATASET_NAME}.returningusers` r
        ON
          e.user_pseudo_id = r.user_pseudo_id
        WHERE
          e.event_timestamp <= r.ts_24hr_after_first_engagement
        )
    SELECT
      user_pseudo_id,
      SUM(IF(event_name = 'user_engagement', 1, 0)) AS cnt_user_engagement,
      SUM(IF(event_name = 'level_start_quickplay', 1, 0)) AS cnt_level_start_quickplay,
      SUM(IF(event_name = 'level_end_quickplay', 1, 0)) AS cnt_level_end_quickplay,
      SUM(IF(event_name = 'level_complete_quickplay', 1, 0)) AS cnt_level_complete_quickplay,
      SUM(IF(event_name = 'level_reset_quickplay', 1, 0)) AS cnt_level_reset_quickplay,
      SUM(IF(event_name = 'post_score', 1, 0)) AS cnt_post_score,
      SUM(IF(event_name = 'spend_virtual_currency', 1, 0)) AS cnt_spend_virtual_currency,
      SUM(IF(event_name = 'ad_reward', 1, 0)) AS cnt_ad_reward,
      SUM(IF(event_name = 'challenge_a_friend', 1, 0)) AS cnt_challenge_a_friend,
      SUM(IF(event_name = 'completed_5_levels', 1, 0)) AS cnt_completed_5_levels,
      SUM(IF(event_name = 'use_extra_steps', 1, 0)) AS cnt_use_extra_steps,
    FROM
      events_first24hr
    GROUP BY
      1
      );

    SELECT
      *
    FROM
      `{PROJECT_ID}.{BQ_DATASET_NAME}.user_aggregate_behavior`
    LIMIT 10
"""
# run the query
run_bq_query(create_behavioral_data_query)

### 合并标签数据和行为数据

现在，在`user_pseudo_id`字段上连接客户流失标签数据和行为数据。 

此操作将创建一个名为`train`的视图，用于进一步训练。

In [ ]:
combine_data_query = f"""
    CREATE OR REPLACE VIEW `{PROJECT_ID}.{BQ_DATASET_NAME}.train` AS (

      SELECT
        ret.user_pseudo_id,
        IFNULL(beh.cnt_user_engagement, 0) AS cnt_user_engagement,
        IFNULL(beh.cnt_level_start_quickplay, 0) AS cnt_level_start_quickplay,
        IFNULL(beh.cnt_level_end_quickplay, 0) AS cnt_level_end_quickplay,
        IFNULL(beh.cnt_level_complete_quickplay, 0) AS cnt_level_complete_quickplay,
        IFNULL(beh.cnt_level_reset_quickplay, 0) AS cnt_level_reset_quickplay,
        IFNULL(beh.cnt_post_score, 0) AS cnt_post_score,
        IFNULL(beh.cnt_spend_virtual_currency, 0) AS cnt_spend_virtual_currency,
        IFNULL(beh.cnt_ad_reward, 0) AS cnt_ad_reward,
        IFNULL(beh.cnt_challenge_a_friend, 0) AS cnt_challenge_a_friend,
        IFNULL(beh.cnt_completed_5_levels, 0) AS cnt_completed_5_levels,
        IFNULL(beh.cnt_use_extra_steps, 0) AS cnt_use_extra_steps,
        ret.user_first_engagement,
        ret.month,
        ret.julianday,
        ret.dayofweek,
        ret.churned
      FROM
        `{PROJECT_ID}.{BQ_DATASET_NAME}.returningusers` ret
      LEFT OUTER JOIN 
        `{PROJECT_ID}.{BQ_DATASET_NAME}.user_aggregate_behavior` beh
      ON
        ret.user_pseudo_id = beh.user_pseudo_id
      WHERE ret.bounced = 0
      );

    SELECT
      *
    FROM
      `{PROJECT_ID}.{BQ_DATASET_NAME}.train`
    LIMIT 10
"""
# run the query
run_bq_query(combine_data_query)

使用BQML训练逻辑回归模型

BQML提供了在BigQuery上使用SQL语法在表格数据上训练机器学习模型的功能，包括分类、回归、预测和矩阵分解。BQML利用了BigQuery的可伸缩基础设施，因此您无需为训练或批处理设置额外的基础设施。

现在，使用BQML在创建的训练数据上训练一个逻辑回归模型。为此，您可以使用BigQuery的SQL语言中的`CREATE OR REPLACE MODEL`语句，并添加以下选项：

* `MODEL_TYPE`：指定用于训练模型的机器学习算法。了解其他[可用的模型类型](https://cloud.google.com/bigquery/docs/reference/standard-sql/bigqueryml-syntax-create#model_type)。

* `INPUT_LABEL_COLS`：指定训练数据中标签列的名称。

* `MODEL_REGISTRY`：指定模型注册的目的地。目前，'VERTEX_AI'是唯一支持的模型注册目的地。要了解更多信息，请参阅[MLOps with BigQuery ML and Vertex AI](https://cloud.google.com/bigquery/docs/create_vertex#register_a_model_to_the)。

* `VERTEX_AI_MODEL_VERSION_ALIASES`：要注册模型的Vertex AI模型别名。只有在`MODEL_REGISTRY`设置为'VERTEX_AI'时才能设置。要了解更多信息，请参阅[添加Vertex AI模型别名](https://cloud.google.com/bigquery/docs/create_vertex#add_a_model_alias)。

了解更多关于在[BQML](https://cloud.google.com/bigquery/docs/bqml-introduction)中训练模型的信息。

请注意，模型名称也遵循与BigQuery中的表格相同的[规则](https://cloud.google.com/bigquery/docs/reference/standard-sql/bigqueryml-syntax-create#model_name)。

In [ ]:
# set the model name
BQ_MODEL_NAME = "churn_model_unique"  # @param {type:"string"}

# define the query for creating and training the BQML model
# remove the id columns user_first_engagement, user_pseudo_id
# for training
train_model_query = f"""
    CREATE OR REPLACE MODEL `{PROJECT_ID}.{BQ_DATASET_NAME}.{BQ_MODEL_NAME}`

    OPTIONS(
      MODEL_TYPE="LOGISTIC_REG",
      INPUT_LABEL_COLS=["churned"],
      MODEL_REGISTRY="VERTEX_AI",
      VERTEX_AI_MODEL_VERSION_ALIASES=['logistic_reg', 'churn_prediction']
    ) AS
  
    SELECT
      * EXCEPT(user_first_engagement, user_pseudo_id)
    FROM
      `{PROJECT_ID}.{BQ_DATASET_NAME}.train`
"""
# run the query
run_bq_query(train_model_query)

获取评估分数

训练完成后，获取训练模型的评估指标分数。

以下单元格应该显示以下字段的结果：

| 精确度 | 召回率 | 准确性 | F1 分数 | 对数损失 | ROC 曲线下面积 |
| -------- | ------- | ------- | ------- | ------- | ------- |

了解有关**ML.EVALUATE**函数返回的[度量标准的更多信息](https://cloud.google.com/bigquery/docs/reference/standard-sql/bigqueryml-syntax-evaluate#mlevaluate_output)。

In [ ]:
# define the query
evaluate_model_query = f"""
    SELECT
      *
    FROM
      ML.EVALUATE(MODEL `{PROJECT_ID}.{BQ_DATASET_NAME}.{BQ_MODEL_NAME}`)
"""

# run the query
run_bq_query(evaluate_model_query)

## 生成带有解释的批处理预测

在训练数据中的一些样本上使用BQML进行批处理预测。每个用户流失的概率都显示在“probability”列中，预测的标签位于“predicted_churn”列下。

此外，您还可以使用`ML.EXPLAIN_PREDICT()`函数生成以特征归因形式的解释。这样您就可以解释每个预测中贡献最大的特征。

返回的输出包括以下附加列：

- `predicted_churned`：预测标签值（流失）的列。

- `probability`：预测标签类的概率。

- `top_feature_attributions`：包含顶部 k 个特征对最终预测的归因的STRUCTs数组。

    - `top_feature_attributions.feature`：特征名称。
    
    - `top_feature_attributions.attribution`：特征对最终预测的贡献。
    
- `baseline_prediction_value`：对于线性模型，baseline_prediction_value是模型的截距。

- `prediction_value`：预测类的对数几率（log-odds）值。

- `approximation_error`：用于解释所使用算法的近似误差。在当前的逻辑回归模型中，没有近似误差，该字段始终为0。了解更多关于[近似误差](https://cloud.google.com/bigquery/docs/reference/standard-sql/bigqueryml-syntax-explain-predict#mlexplain_predict_output)的信息。

了解更多关于[**ML.EXPLAIN_PREDICT**](https://cloud.google.com/bigquery-ml/docs/reference/standard-sql/bigqueryml-syntax-xai-overview)的信息。

注意：您可以简单地使用[**ML.PREDICT**](https://cloud.google.com/bigquery/docs/reference/standard-sql/bigqueryml-syntax-predict#mlpredict_function)来获取没有解释的预测。

In [ ]:
# write sql query for predictions with explanations
explain_predict_query = f"""
    SELECT
      *
    FROM
      ML.EXPLAIN_PREDICT(
        MODEL `{PROJECT_ID}.{BQ_DATASET_NAME}.{BQ_MODEL_NAME}`,
        (
            SELECT * FROM `{PROJECT_ID}.{BQ_DATASET_NAME}.train` LIMIT 10
        )
    )
"""
# run the sql query
run_bq_query(explain_predict_query)

从模型注册表中获取模型

先前使用`MODEL_REGISTRY`设置为`VERTEX_AI`的`CREATE MODEL`语句在Vertex AI模型注册表中创建了一个模型。

使用模型名称获取模型资源。

In [ ]:
model = vertex_ai.Model(model_name=BQ_MODEL_NAME)

print(model.gca_resource)

## 创建部署的端点

虽然BQML支持批量预测，但它不适用于需要低延迟预测以及可能频繁请求的实时预测。因此，将BQML模型部署到端点可以让您进行在线预测。

创建一个Vertex AI端点来部署BQML模型。

In [ ]:
ENDPOINT_NAME = "churn-model-endpoint-unique"  # @param {type:"string"}

endpoint = vertex_ai.Endpoint.create(
    display_name=ENDPOINT_NAME,
    project=PROJECT_ID,
    location=REGION,
)

print("Endpoint display name:", endpoint.display_name)
print("Endpoint resource name:", endpoint.resource_name)

部署模型到端点

使用 [**deploy**](https://cloud.google.com/python/docs/reference/aiplatform/latest/google.cloud.aiplatform.Model#google_cloud_aiplatform_Model_deploy) 方法将模型资源部署到已创建的端点资源。

当您的BQML模型在Vertex AI模型注册表中注册，并且是支持可解释性人工智能的模型类型时，您可以在将模型部署到端点时启用可解释性人工智能。在本教程中，您将不使用可解释性人工智能部署模型。

了解更多关于[将BQML模型部署到启用可解释性人工智能的Vertex AI端点](https://cloud.google.com/bigquery/docs/vertex-xai#enable_explainable_ai_in)。

In [ ]:
# deploying the model to the endpoint may take 10-15 minutes
model.deploy(endpoint=endpoint)

为在线预测创建有效载荷

查询训练数据以获取一些样本（比如2条记录），以便作为在线请求发送到终端。实例有效载荷需格式化为一组键值对列表，其中键代表一个特征。

In [ ]:
# query to select two instances for testing
test_df = run_bq_query(
    f"""
    SELECT 
    * except(user_first_engagement, user_pseudo_id, churned) 
    FROM `{PROJECT_ID}.{BQ_DATASET_NAME}.train` 
    LIMIT 2
    """
)
# convert the dataframe to json instances to send
df_sample_requests_list = json.loads(test_df.to_json(orient="records"))

# display the data
test_df

将在线请求发送到端点

使用端点的 **predict** 方法，并发送要预测的实例列表。

返回响应中的预测包括以下内容：

- `churned_probs`: 每个类别的概率。
- `predicted_churned`: 预测的类别。
- `churned_values`: 类别的可能值（1 代表流失，0 代表未流失）。

In [ ]:
# send prediction request to the endpoint
prediction = endpoint.predict(df_sample_requests_list)
# print the predictions
print(prediction.predictions)

## 清理

要清理此项目中使用的所有 Google Cloud 资源，您可以删除用于教程的 [Google Cloud 项目](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects)。

否则，您可以删除在本教程中创建的各个资源：

- Vertex AI 端点
- Vertex AI 模型（无法从 Vertex AI 中删除 BQML 模型，需要从 BigQuery 中删除）
- BigQuery 数据集
- Cloud Storage 存储桶

In [ ]:
# set True to delete the dataset
delete_dataset = False
# set True to delete the bucket
delete_bucket = False

# undeploy model from the endpoint
endpoint.undeploy_all()
# # delete the endpoint
endpoint.delete()

# delete BigQuery dataset
if delete_dataset or os.getenv("IS_TESTING"):
    ! bq rm -r -f $PROJECT_ID:$BQ_DATASET_NAME
# delete the Cloud Storage bucket
if delete_bucket or os.getenv("IS_TESTING"):
    ! gsutil -m rm -r $BUCKET_URI